# Pulse Client High-Level API Examples

This notebook demonstrates how to use the high-level `Analyzer` interface with built-in `Process` classes to orchestrate complex workflows.


In [ ]:
!pip install pulse-sdk

In [ ]:
from pulse.analysis.analyzer import Analyzer
from pulse.analysis.processes import (
    ThemeGeneration,
    ThemeAllocation,
    ThemeExtraction,
    SentimentProcess,
    Cluster,
)

## 1. Theme Generation, Allocation, and Extraction
Generate themes, allocate them to texts, and extract representative elements.


In [ ]:
texts = [
    "I love the new interface",
    "The app crashes on launch",
    "Customer support was helpful",
    "Slow performance after update",
]

az = Analyzer(
    dataset=texts,
    processes=[
        ThemeGeneration(min_themes=2, max_themes=4),
        ThemeAllocation(single_label=False, threshold=0.0),
        ThemeExtraction(),
    ],
    fast=True,
    cache_dir=".pulse_cache",
)

res = az.run()
print("Detected themes:", res.theme_generation.themes)
df_alloc = res.theme_allocation.to_dataframe()
display(df_alloc)
df_extr = res.theme_extraction.to_dataframe()
display(df_extr)

## 2. Sentiment Analysis
Run sentiment classification as part of the same workflow.


In [ ]:
az2 = Analyzer(
    dataset=texts,
    processes=[SentimentProcess()],
    fast=True,
)
res2 = az2.run()
print(res2.sentiment.summary())
display(res2.sentiment.to_dataframe())

## 3. Clustering with Similarity
Compute similarity matrix and explore clustering.


In [ ]:
az3 = Analyzer(
    dataset=texts,
    processes=[Cluster(fast=True)],
    fast=True,
)
res3 = az3.run()
labels_km = res3.cluster.kmeans(n_clusters=2)
print("KMeans labels:", labels_km)
ax = res3.cluster.plot_scatter()
ax.set_title("PCA Scatter of Text Similarity")

## 4. Caching Demonstration
Re-run the same processes to see cache hits.


In [ ]:
res_cached = az.run()  # Should use cache
print("Cache hit for themes:", res_cached.theme_generation.themes)